# --- Day 6: Tuning Trouble ---

The preparations are finally complete; you and the Elves leave camp on foot and begin to make your way toward the **star** fruit grove.

As you move through the dense undergrowth, one of the Elves gives you a handheld device. He says that it has many fancy features, but the most important one to set up right now is the **communication system**.

However, because he's heard you have significant experience dealing with signal-based systems, he convinced the other Elves that it would be okay to give you their one malfunctioning device - surely you'll have no problem fixing it.

As if inspired by comedic timing, the device emits a few colorful sparks.

To be able to communicate with the Elves, the device needs to lock on to **their signal**. The signal is a series of seemingly-random characters that the device receives one at a time.

To fix the communication system, you need to add a subroutine to the device that detects a **start-of-packet marker** in the datastream. In the protocol being used by the Elves, the start of a packet is indicated by a sequence of **four characters that are all different**.

The device will send your subroutine a datastream buffer (your puzzle input); your subroutine needs to identify the first position where the four most recently received characters were all different. Specifically, it needs to report the number of characters from the beginning of the buffer to the end of the first such four-character marker.

For example, suppose you receive the following datastream buffer:

`mjqjpqmgbljsphdztnvjfqwrcgsmlb`

After the first three characters (`mjq`) have been received, there haven't been enough characters received yet to find the marker. The first time a marker could occur is after the fourth character is received, making the most recent four characters `mjqj`. Because `j` is repeated, this isn't a marker.

The first time a marker appears is after the **seventh** character arrives. Once it does, the last four characters received are `jpqm`, which are all different. In this case, your subroutine should report the value `7`, because the first start-of-packet marker is complete after 7 characters have been processed.

Here are a few more examples:

- `bvwbjplbgvbhsrlpgdmjqwftvncz`: first marker after character `5`
- `nppdvjthqldpwncqszvftbrmjlhg`: first marker after character `6`
- `nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg`: first marker after character `10`
- `zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw`: first marker after character `11`

**How many characters need to be processed before the first start-of-packet marker is detected?**

In [1]:
#imports
import pandas as pd

In [2]:
input = 'input.txt'
input = 'testinput.txt'

In [3]:
def readinput(file):
    with open(file) as f:
        #lines = f.readlines()
        lines = f.read().splitlines() 
    return lines

input = readinput(input)

In [4]:
#input = input[0]
input

['R 4', 'U 4', 'L 3', 'D 1', 'R 4', 'D 1', 'L 5', 'R 2']

In [5]:
def dostep(isx:int, isy:int, direction:str):
    if direction == 'R':
        isx += 1
        position = [isx, isy]
    if direction == 'L':
        isx -= 1
        position = [isx, isy]
    if direction == 'U':
        isy += 1
        position = [isx, isy]
    if direction == 'D':
        isy -= 1
        position = [isx, isy]
    return position

In [6]:
def mapcreation(stepslist):
    isnow = [0,0]
    maxu = 0
    maxd = 0
    maxl = 0
    maxr = 0
    for step in stepslist:
        direction, steps = step.split(' ')
        steps = int(steps)
        for i in range(steps):
            isx, isy = isnow
            isnow = dostep(isx, isy, direction)
            isx, isy = isnow
            
            if isx > maxr:
                maxr = isx
            elif isx < maxl:
                maxl = isx
            if isy > maxu:
                maxu = isy
            elif isy < maxd:
                maxd = isy

        
    maxx = (maxl*-1)+maxr
    maxy = (maxd*-1)+maxu
    return maxx, maxy

sizex, sizey = mapcreation(input)
print(f'X: {sizex}\nY: {sizey}')

X: 5
Y: 4


In [7]:
def followhead(hpos, tpos):
    
    hisx, hisy = hpos
    tisx, tisy = tpos
    
    xdiff = hisx-tisx
    ydiff = hisy-tisy
    
    if xdiff <= 0:
        xdif = xdiff*-1
    else:
        xdif = xdiff
    if ydiff <= 0:
        ydif = ydiff*-1
    else:
        ydif = ydiff
        
    #print(f'x-dif:{xdif}')
    #print(f'y-dif:{ydif}')
    
    # movevertical
    if xdif >= 2 and ydif >=1:
        if xdiff < 0:
            tisx -=1
        elif xdiff > 0:
            tisx += 1
        if ydiff < 0:
            tisy -=1
        elif ydiff > 0:
            tisy += 1
        #tisx += 1
        #tisy += 1
    elif xdif >= 1 and ydif >=2:
        if xdiff < 0:
            tisx -=1
        elif xdiff > 0:
            tisx += 1
        if ydiff < 0:
            tisy -=1
        elif ydiff > 0:
            tisy += 1
        
        position = [tisx, tisy]
        return position
    
    # move horizotal
    if xdif > 1 and ydif == 0:
        if xdiff < 0:
            tisx -=1
        elif xdiff > 0:
            tisx += 1
        position = [tisx, tisy]
        return position
    
    # move vertical
    if ydif > 1 and xdif == 0:
        if ydiff < 0:
            tisy -= 1
        elif ydiff > 0:
            tisy += 1
        position = [tisx, tisy]
        return position
    
    #print('fail')
    position = [tisx, tisy]
    return tpos

In [8]:
def move(stepslist, maxx, maxy):
    dfmap = pd.DataFrame()
    ispos = {}
    ispos['s'] = [0,0]
    ispos['H'] = [0,0]
    ispos['T'] = [0,0]
    ispos['x'] = [0,0] # X = #
    washere = []
    
    dfmap = [{0: [ispos['H'], ispos['T'], ispos['s'], ispos['x']]}]
    dfmap = pd.DataFrame.from_dict(dfmap)
    
    
    for steplist in stepslist:
        print(steplist)
        direction, steps = steplist.split(' ')
        steps = int(steps)
        # every single step if move
        for i in range(steps):
            # is position form H
            hisx, hisy = ispos['H']
            tisx, tisy = ispos['T']
            # new position from H
            ispos['H'] = dostep(hisx, hisy, direction)
            print(f'H: {[hisx, hisy]} -> {ispos["H"]}')
            ispos['T'] = followhead(ispos['H'], ispos['T'])
            print(f'T: {[tisx, tisy]} -> {ispos["T"]}')
            washere.append(ispos['T'])
            
    #washere = list(set(washere))
    return washere

xxx = move(input, sizex, sizey)

R 4
H: [0, 0] -> [1, 0]
T: [0, 0] -> [0, 0]
H: [1, 0] -> [2, 0]
T: [0, 0] -> [1, 0]
H: [2, 0] -> [3, 0]
T: [1, 0] -> [2, 0]
H: [3, 0] -> [4, 0]
T: [2, 0] -> [3, 0]
U 4
H: [4, 0] -> [4, 1]
T: [3, 0] -> [3, 0]
H: [4, 1] -> [4, 2]
T: [3, 0] -> [4, 1]
H: [4, 2] -> [4, 3]
T: [4, 1] -> [4, 2]
H: [4, 3] -> [4, 4]
T: [4, 2] -> [4, 3]
L 3
H: [4, 4] -> [3, 4]
T: [4, 3] -> [4, 3]
H: [3, 4] -> [2, 4]
T: [4, 3] -> [4, 3]
H: [2, 4] -> [1, 4]
T: [4, 3] -> [4, 3]
D 1
H: [1, 4] -> [1, 3]
T: [4, 3] -> [3, 3]
R 4
H: [1, 3] -> [2, 3]
T: [3, 3] -> [3, 3]
H: [2, 3] -> [3, 3]
T: [3, 3] -> [3, 3]
H: [3, 3] -> [4, 3]
T: [3, 3] -> [3, 3]
H: [4, 3] -> [5, 3]
T: [3, 3] -> [4, 3]
D 1
H: [5, 3] -> [5, 2]
T: [4, 3] -> [4, 3]
L 5
H: [5, 2] -> [4, 2]
T: [4, 3] -> [4, 3]
H: [4, 2] -> [3, 2]
T: [4, 3] -> [4, 3]
H: [3, 2] -> [2, 2]
T: [4, 3] -> [4, 3]
H: [2, 2] -> [1, 2]
T: [4, 3] -> [4, 3]
H: [1, 2] -> [0, 2]
T: [4, 3] -> [4, 3]
R 2
H: [0, 2] -> [1, 2]
T: [4, 3] -> [4, 3]
H: [1, 2] -> [2, 2]
T: [4, 3] -> [4, 3]


In [9]:
nr = 0
singlepos = []
for pos in xxx:
    if pos not in singlepos:
        singlepos.append(pos)
        nr += 1
        
nr

8

# --- Part Two ---

Your device's communication system is correctly detecting packets, but still isn't working. It looks like it also needs to look for **messages**.

A **start-of-message** marker is just like a start-of-packet marker, except it consists of **14 distinct characters** rather than 4.

Here are the first positions of start-of-message markers for all of the above examples:

- `mjqjpqmgbljsphdztnvjfqwrcgsmlb`: first marker after character `19`
- `bvwbjplbgvbhsrlpgdmjqwftvncz`: first marker after character `23`
- `nppdvjthqldpwncqszvftbrmjlhg`: first marker after character `23`
- `nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg`: first marker after character `29`
- `zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw`: first marker after character `26`

**How many characters need to be processed before the first start-of-message marker is detected?**

In [10]:
x = findsignal(input, 14)

NameError: name 'findsignal' is not defined

In [ ]:
x